In [27]:
import pandas as pd
import numpy as np
import re
import unicodedata
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import nltk

In [2]:
def clean_text(text, pattern="[^a-zA-Z ]"):
    cleaned_text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore')
    cleaned_text = re.sub(pattern, "", cleaned_text.decode("utf-8"), flags=re.UNICODE)
    cleaned_text = u' '.join(cleaned_text.lower().split())
    return cleaned_text

In [3]:
df=pd.read_csv('Experiencias Telcel_23 septiembre 2021_09.13.csv')

In [4]:
df=df.iloc[2:]

In [12]:
s=df[df['Q6_13_TEXT'].notna()]['Q6_13_TEXT']

In [15]:
s.name='¿Dónde has escuchado o visto información sobre Experiencias Telcel?'

In [13]:
s1=df[df['Q7_13_TEXT'].notna()]['Q7_13_TEXT']

In [16]:
s1.name='¿Qué tipos de premios te gustaría ganar?'

In [14]:
s2=df[df['Q9_8_TEXT'].notna()]['Q9_8_TEXT']

In [17]:
s2.name='¿Por qué razón no has participado en el programa de premios “Experiencia Telcel"?'

In [22]:
s.to_csv('Donde.csv',encoding='latin-1')

In [23]:
s1.to_csv('Premios.csv',encoding='latin-1')

In [26]:
s2.to_csv('Por_que.csv',encoding='utf-16-le')

In [36]:
df.head()

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,Q4,Q5,Q6,Q6_13_TEXT,Q7,Q7_13_TEXT,Q8,Q9,Q9_8_TEXT,Q10
2,2021-09-17 10:28:53,2021-09-17 10:29:31,Survey Preview,NaN,100,37,True,2021-09-17 10:29:31,R_2EsCT7y1oIwGA2g,NaN,...,NaN,No,NaN,NaN,NaN,NaN,NaN,\tNo me gusta el premio,NaN,NaN
3,2021-09-17 10:32:44,2021-09-17 10:33:01,Survey Preview,NaN,100,16,True,2021-09-17 10:33:01,R_3hlpRiDMpv4FHPr,NaN,...,Menor de 18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-09-17 10:33:06,2021-09-17 10:33:35,Survey Preview,NaN,100,28,True,2021-09-17 10:33:35,R_RlChoneS7XItaBr,NaN,...,26 - 35,Sí,\tTelevisión,NaN,\tPases para conciertos y eventos exclusivos,NaN,<,NaN,NaN,NaN
5,2021-09-20 09:55:22,2021-09-20 10:19:17,IP Address,200.68.129.118,100,1435,True,2021-09-20 10:19:18,R_1Hpgdq5Pn9fJqn4,NaN,...,Menor de 18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2021-09-20 10:19:31,2021-09-20 10:22:53,IP Address,200.68.129.118,100,202,True,2021-09-20 10:22:53,R_32QPvOR6ucZ7VYD,NaN,...,26 - 35,Sí,"\tTelevisión ,\tRedes Sociales ,\tInternet ...",NaN,"\tSmartphones, Tablets, dispositivos IoT, gad...",NaN,prueba,NaN,NaN,NaN


In [28]:
df['Q5'].value_counts()

No    2877
Sí     410
Name: Q5, dtype: int64

In [29]:
dfg=df[df['Q8'].notna()]

In [30]:
dfng=df[df['Q10'].notna()]

In [31]:
a=dfg[['Q8']].append(dfng['Q10'])

In [34]:
c=dfg['ExternalReference'].append(dfng['ExternalReference'])

In [36]:
c.to_csv('Lineas.csv',index=False)

In [37]:
c

4              NaN
6              NaN
8              NaN
15      9211846163
19      4981162396
           ...    
3325    8333263939
3327    5522988795
3328    5545852367
3329    5540320424
3330    4771315354
Name: ExternalReference, Length: 3286, dtype: object

In [42]:
b=a.copy()

In [43]:
def tokestem(x):
    stemmer = SnowballStemmer('spanish')
    a=word_tokenize(x)
    a=[stemmer.stem(i) for i in a]
    return a

In [44]:
a=a.str.replace("\n", "").map(clean_text).map(tokestem).str.join(" ")

In [45]:
a

4                                      
6                                 prueb
8                                 kwkwk
15                    referent al premi
19                                no se
                     ...               
3325                                 no
3327                               avis
3328                       equip telcel
3329    es una telefoni car considerenl
3330         pues cualqui premi es buen
Length: 3286, dtype: object

In [46]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [47]:
sw=stopwords.words('spanish')

In [48]:
stemmer = SnowballStemmer('spanish')

In [49]:
sw=list(set([stemmer.stem(i) for i in sw]))

In [50]:
sw=list(pd.Series(sw).map(clean_text))

In [813]:
sw.remove('nad')

In [814]:
sw.append('gust')

In [815]:
sw.remove('no')

In [816]:
sw.append('nad')

In [817]:
sw.append('consum')

In [818]:
sw.append('no')

In [819]:
sw.append('bien')

In [820]:
sw.append('dat')

In [91]:
a=a.str.replace('telefon','equip')

In [92]:
vect = TfidfVectorizer(min_df=20, ngram_range=(1,3), stop_words=list( sw))

In [93]:
X=vect.fit_transform(a)

In [94]:
X.shape

(3286, 80)

In [95]:
from sklearn.decomposition import TruncatedSVD

# SVD represent documents and terms in vectors 
svd_model = TruncatedSVD(n_components=7, algorithm='randomized', n_iter=100, random_state=122)

svd_model.fit(X)

len(svd_model.components_)

7

In [96]:
terms = vect.get_feature_names()

for i, comp in enumerate(svd_model.components_):
    terms_comp = zip(terms, comp)
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:7]
    print("Topic "+str(i)+": ")
    for t in sorted_terms:
        print(t[0])
        print(" ")

Topic 0: 
premi
 
equip
 
tiemp
 
air
 
tiemp air
 
celular
 
gratis
 
Topic 1: 
equip
 
tiemp
 
tiemp air
 
air
 
gratis
 
plan
 
celular
 
Topic 2: 
tiemp
 
air
 
tiemp air
 
gratis
 
celular
 
meg
 
etc
 
Topic 3: 
celular
 
sald
 
gratis
 
recarg
 
meg
 
equip celular
 
plan
 
Topic 4: 
sald
 
promocion
 
gratis
 
meg
 
gan
 
plan
 
recarg
 
Topic 5: 
conozc
 
promocion
 
gan
 
ide
 
telcel
 
dinam
 
experient
 
Topic 6: 
gan
 
facil
 
oportun
 
particip
 
celul
 
meg
 
trivi
 


In [97]:
aux=pd.concat([pd.Series(np.argmax(svd_model.transform(X),axis=1)),pd.Series(np.sum(svd_model.transform(X),axis=1)),b.reset_index(drop=True)],axis=1)

In [98]:
aux

,0,1,2
0,0,0.000000,<
1,0,0.000000,prueba
2,0,0.000000,kwkwk
3,0,-0.286408,Referente al premio
4,0,0.000000,No se
...,...,...,...
3281,0,0.000000,No
3282,0,0.000000,Avisos
3283,1,0.584008,Equipos telcel
3284,0,0.000000,Es una telefonía cara. Considerenlo


In [99]:
aux.columns=['Tema','aux','Texto']

In [100]:
aux['Tema1']=pd.Series(map(lambda x,y: 7 if y==0 else x, aux['Tema'],aux['aux']))

In [101]:
aux['Tema1'].value_counts()

7    838
4    589
5    552
0    378
6    369
1    228
2    196
3    136
Name: Tema1, dtype: int64

In [118]:
aux[aux['Tema1']==3].sample(20)

,Tema,aux,Texto,Tema1
1653,3,0.913901,Celulares,3
2298,3,0.913901,Celulares,3
1014,3,0.440350,Celulares premios en efectivo o paquetes sin l...,3
2407,3,0.565125,"Algunos minutos , bocinas , celulares , cursos...",3
2443,3,0.820043,$ dinero $ viajes $ equipos celulares gratis,3
1610,3,0.549208,"Celulares, o accesorios o mochilas",3
866,3,0.829094,Teléfonos celulares,3
2491,3,0.913901,Celulares,3
1044,3,0.913901,Celulares,3
1226,3,0.680405,Celulares gratis de gama alta,3


In [119]:
aux['Tema1']=aux['Tema1'].replace(3,1)

In [120]:
aux['Tema1']=aux['Tema1'].replace(4,2)

In [121]:
aux

,Tema,aux,Texto,Tema1
0,0,0.000000,<,7
1,0,0.000000,prueba,7
2,0,0.000000,kwkwk,7
3,0,-0.286408,Referente al premio,0
4,0,0.000000,No se,7
...,...,...,...,...
3281,0,0.000000,No,7
3282,0,0.000000,Avisos,7
3283,1,0.584008,Equipos telcel,1
3284,0,0.000000,Es una telefonía cara. Considerenlo,7


In [122]:
aux['Nombre']=aux['Tema1'].replace({0:'Premios llamativos',1:'Equipos/Accesorios',2:'Promociones en servicio',5:'No conozco para opinar',6:'Posibilidades de ganar/Trivias',7:'Otros'})

In [123]:
aux[['Texto','Nombre']]

,Texto,Nombre
0,<,Otros
1,prueba,Otros
2,kwkwk,Otros
3,Referente al premio,Premios llamativos
4,No se,Otros
...,...,...
3281,No,Otros
3282,Avisos,Otros
3283,Equipos telcel,Equipos/Accesorios
3284,Es una telefonía cara. Considerenlo,Otros


In [124]:
dfg['Participó']=1

<ipython-input-124-247c4e7d909c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfg['Participó']=1


In [125]:
dfng['Participó']=0

<ipython-input-125-15f1703c6d36>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfng['Participó']=0


In [126]:
c=dfg['Participó'].append(dfng['Participó'])

In [127]:
c

4       1
6       1
8       1
15      1
19      1
       ..
3325    0
3327    0
3328    0
3329    0
3330    0
Name: Participó, Length: 3286, dtype: int64

In [846]:
c1=dfg1[['Encuesta','Gusta']].append(dfng1[['Encuesta','Gusta']])

In [847]:
c=c.append(c1)

In [787]:
c['Texto']=b

In [848]:
c

,Encuesta,Gusta
4,Nacional,1
7,Nacional,1
8,Nacional,1
10,Nacional,1
11,Nacional,1
...,...,...
1057,Roaming,0
1058,Roaming,0
1059,Roaming,0
1060,Roaming,0


In [128]:
aux=pd.concat([aux,c.reset_index(drop=True)],axis=1)

In [140]:
aux[['Texto','Nombre','Participó']].to_csv('Texto_Exp.csv',index=False)

In [147]:
df[df['Q6_13_TEXT'].notna()]['Q6_13_TEXT']

308     Ppr los correos que mandan de promociones
761                                         Buzón
2279                                        Email
2306                          Correo electrónico 
2635                                          Sms
2652                               Un sitio porno
Name: Q6_13_TEXT, dtype: object

In [149]:
df.columns

Index(['StartDate', 'EndDate', 'Status', 'IPAddress', 'Progress',
       'Duration (in seconds)', 'Finished', 'RecordedDate', 'ResponseId',
       'RecipientLastName', 'RecipientFirstName', 'RecipientEmail',
       'ExternalReference', 'LocationLatitude', 'LocationLongitude',
       'DistributionChannel', 'UserLanguage', 'Q2_Browser', 'Q2_Version',
       'Q2_Operating System', 'Q2_Resolution', 'Q3', 'Q4', 'Q5', 'Q6',
       'Q6_13_TEXT', 'Q7', 'Q7_13_TEXT', 'Q8', 'Q9', 'Q9_8_TEXT', 'Q10'],
      dtype='object')

In [154]:
df[df['Q6'].notna()]['Q6'].value_counts()

\tMi Telcel                                                                                                                                                                         166
\tRedes Sociales                                                                                                                                                                     33
\tMi Telcel,\tTelcel.com                                                                                                                                                            32
\tRedes Sociales ,\tMi Telcel                                                                                                                                                       31
\tInternet                                                                                                                                                                           15
\tInternet ,\tMi Telcel                                                  